In [6]:
import os
import json

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL')


In [7]:
message = "Hello, I am learning AI stuff"
messages = [{"role": "user", "content": message}]
messages

[{'role': 'user', 'content': 'Hello, I am learning AI stuff'}]

Only for testing local llm

In [18]:

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

# response = ollama.chat.completions.create(model="gpt-oss:latest", messages=[{"role": "user", "content": "Tell me a fun fact"}])

# response.choices[0].message.content

In [9]:
links = fetch_website_links("http://innoes.co.in")
links

['index.html',
 'index.html',
 'tab-iaq.html',
 'energyaudit.html',
 'trainings.html',
 'contact.html',
 'https://api.whatsapp.com/send?phone=919962228968&text=Interested%20to%20contact%20you%20in%20Whatsapp',
 '#targetToScrollDownTo',
 'https://www.facebook.com/Innovative-Engineering-Solution-105606864112703/',
 'https://www.instagram.com/innovaive/',
 'https://twitter.com/InnovativeEngi5',
 'https://www.linkedin.com/in/innovative-engineering-559654190/',
 'index.html',
 'tab-iaq.html',
 'energyaudit.html',
 'trainings.html',
 'contact.html',
 'https://justsee.co.in']

In [10]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "http://innoes.co.in/tab-iaq.html"},
        {"type": "contact page", "url": "http://innoes.co.in/contact.html"}
    ]
}
"""

In [13]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [17]:
print(get_links_user_prompt("http://innoes.co.in"))


Here is the list of links on the website http://innoes.co.in -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

index.html
index.html
tab-iaq.html
energyaudit.html
trainings.html
contact.html
https://api.whatsapp.com/send?phone=919962228968&text=Interested%20to%20contact%20you%20in%20Whatsapp
#targetToScrollDownTo
https://www.facebook.com/Innovative-Engineering-Solution-105606864112703/
https://www.instagram.com/innovaive/
https://twitter.com/InnovativeEngi5
https://www.linkedin.com/in/innovative-engineering-559654190/
index.html
tab-iaq.html
energyaudit.html
trainings.html
contact.html
https://justsee.co.in


In [19]:
def select_relevant_links(url):
    response = ollama.chat.completions.create(
        model="gpt-oss:latest",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [20]:
select_relevant_links("http://innoes.co.in")

{'links': [{'type': 'home page', 'url': 'https://innoes.co.in/index.html'},
  {'type': 'IAQ service page', 'url': 'https://innoes.co.in/tab-iaq.html'},
  {'type': 'Energy Audit service page',
   'url': 'https://innoes.co.in/energyaudit.html'},
  {'type': 'Trainings service page',
   'url': 'https://innoes.co.in/trainings.html'},
  {'type': 'contact page', 'url': 'https://innoes.co.in/contact.html'},
  {'type': 'whatsapp',
   'url': 'https://api.whatsapp.com/send?phone=919962228968&text=Interested%20to%20contact%20you%20in%20Whatsapp'},
  {'type': 'facebook',
   'url': 'https://www.facebook.com/Innovative-Engineering-Solution-105606864112703/'},
  {'type': 'instagram', 'url': 'https://www.instagram.com/innovaive/'},
  {'type': 'twitter', 'url': 'https://twitter.com/InnovativeEngi5'},
  {'type': 'linkedin',
   'url': 'https://www.linkedin.com/in/innovative-engineering-559654190/'},
  {'type': 'partner', 'url': 'https://justsee.co.in'}]}

In [21]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [24]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
2 days ago
•
44.5k
•
1.39k
black-forest-labs/FLUX.2-dev
Updated
3 days ago
•
171k
•
739
tencent/HunyuanOCR
Updated
3 days ago
•
92.9k
•
526
deepseek-ai/DeepSeek-Math-V2
Updated
3 days ago
•
3.53k
•
485
microsoft/Fara-7B
Updated
1 day ago
•
10.7k
•
306
Browse 1M+ models
Spaces
Running
on
Zero
MCP
726
Z Image Turbo
🏃
726
Generate images from text prompts
Running
on
Zero
290
FLUX.2 [dev]
💻
290
Generate detailed images from text prompts or edit existing images
Running
on
Zero
MCP
Featured
1.4k
Qwen Image Edit Camera Control
🎬
1.4k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
Zero
MCP
Fea

In [25]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [26]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n2 days ago\n•\n44.5k\n•\n1.39k\nblack-forest-labs/FLUX.2-dev\nUpdated\n3 days ago\n•\n171k\n•\n739\ntencent/HunyuanOCR\nUpdated\n3 days ago\n•\n92.9k\n•\n526\ndeepseek-ai/DeepSeek-Math-V2\nUpdated\n3 days ago\n•\n3.53k\n•\n485\nmicrosoft/Fara-7B\nUpdated\n1 day ago\n•\n10.7k\n•\n306\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\n726\nZ Image Turbo\n🏃\

In [29]:
def create_brochure(company_name, url):
    response = ollama.chat.completions.create(
        # model="gpt-4.1-mini",
        model="gpt-oss:latest",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face  
**The AI community building the future**

---

## 1️⃣  Who We Are  
Hugging Face is a global, open‑source hub where researchers, developers, and businesses collaborate on a single, unified AI platform. With **over 1 million models, 250 k+ datasets, and 400 k+ Applications (Spaces)**, we’re the go‑to source for anyone looking to build, share, or deploy state‑of‑the‑art machine‑learning solutions.

| Feature | What It Means |
|---------|---------------|
| **Models** | 1 M+ ready‑to‑use models that span text, image, video, audio, and 3‑D. |
| **Datasets** | 250 k+ curated collections covering every use‑case and modality. |
| **Spaces** | 400 k+ interactive demos and micro‑apps, hosted “for free” and powered by our **Zero‑MCP** inference runtime. |
| **Community** | A worldwide community of researchers, hobbyists, and industry professionals who review, improve, and build upon each other’s work. |

---

## 2️⃣  What We Build  
* **HF Open‑Source Stack** – fast, battle‑tested libraries (e.g., 🤗 Transformers, 🤗 Datasets, 🤗 Tokenizers) that let you experiment locally in minutes.  
* **Enterprise‑Grade Platform** – secure, compliant hosting of models, datasets, and Spaces.  
* **Compute & Inference Services** – pay‑as‑you‑go or subscription compute that scales from a single notebook to a full org.  

---

## 3️⃣  For Customers  
### Enterprise Customers  
* **Team Plans** – start at **$20/user/month** with SSO, audit logs, regional data control, and dedicated support.  
* **Enterprise Contracts** – custom SLAs, enterprise‑grade security, and on‑prem or hybrid deployment options.  
* **Security & Compliance** – SOC 2‑Certified, GDPR‑ready, and enterprise‑ready access controls.  

> *“The Hugging Face platform lets us prototype faster and ship safer AI models at scale.”* – **Data Scientist, Fortune 500**

---

## 4️⃣  For Investors  
* **Rapid Marketplace Growth** – 1 M+ models + 400 k+ Spaces mean a thriving ecosystem of developers, collaborators, and commercial customers.  
* **Strategic Position** – First‑mover in unifying all modalities (text, image, video, audio, 3‑D) on a single open‑source platform.  
* **Scalable Monetization** – Subscription teams, enterprise contracts, and cloud compute APIs create multiple recurring revenue streams.  

---

## 5️⃣  For Talent  
* **Mission‑Driven Culture** – A place where curiosity, openness, and impact guide every decision.  
* **Remote‑First, Inclusive Teams** – Work from anywhere, collaborate with 30 + countries, and join diverse, high‑impact projects.  
* **Open‑Source Leadership** – Contribute to the world’s most popular ML libraries, gain global visibility, and advance your career.  

> **Open roles** (as of now):  
> * ML Engineer – Transformers & Inference  
> * DevOps Engineer – Cloud & SSO  
> * Data Engineer – Datasets & ETL  
> * Product Manager – Enterprise AI Solutions  

More positions and career paths are added frequently—check the **Careers** page on our website.

---

## 📌 How to Join or Learn More  
1. **Explore** – Browse 1 M+ models or 400 k+ Spaces.  
2. **Build** – Use HF libraries in your notebook or deploy via the Spaces UI.  
3. **Collaborate** – Contribute to a model, dataset, or app and let the community review it.  
4. **Scale** – Upgrade to a Team or Enterprise plan, or purchase compute for production inference.  

**Sign up** today and collaborate with the world’s leading AI community. For enterprise inquiries, contact sales through our website.  

---

In [31]:
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        # model="gpt-4.1-mini",
        model="gpt-oss:latest",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## 🚀 The AI community building the future

Hugging Face is the hub where researchers, developers and industry leaders collaborate on the world’s most ambitious machine‑learning projects. From millions of pre‑trained models to massive open datasets and zero‑latency inference, our platform lets you build, share, and scale AI faster than ever.

---

## ✨ Core Offerings

| Feature | What It Gives You |
|---------|------------------|
| **Models** | Browse & host 1M + models (text, image, video, audio, 3D). 100+ of the most popular are updated daily. |
| **Datasets** | 250 k+ curated datasets; create your own private collections with the same tools. |
| **Spaces** | One‑click hosting of AI demos (inference, generation, editing). ZeroGPU, ZeroMCP, and custom compute options. |
| **Docs & Community** | Extensive API docs, tutorials, forums, and a vibrant community of contributors. |

---

## 🌍 Enterprise‑Ready AI

- **Team & Enterprise Hub** – Advanced security, SSO, audit logs, resource groups, token management.  
- **Compute Flexibility** – ZeroGPU quota boosts, custom inference providers, private storage (1 TB/member).  
- **Analytics & Billing** – Unified usage dashboard, spend controls, managed billing, yearly commitments.  
- **Priority Support** – Dedicated account managers and priority help for mission‑critical workloads.

Pricing starts at **$20/user/month** for Team; Enterprise contracts available on request.

---

## 💰 Pricing Tiers

| Tier | Monthly Cost | Highlights |
|------|---------------|------------|
| **Free** | $0 | Unlimited public repos, community access. |
| **PRO** | $9 | 10× private storage, 20× inference credits, 8× ZeroGPU quota, dev‑mode Spaces. |
| **Team** | $20/user | SSO, granular permissions, collaboration tools. |
| **Enterprise** | Custom | Advanced security, SLAs, billing, dedicated support. |

---

## 🤝 Why Organizations Trust Hugging Face

- **Open‑Source Leadership** – We publish core libraries (🤗 transformers, 🤗 datasets, 🤗 accelerate) to the community.  
- **Security & Compliance** – ISO, SOC 2, GDPR ready; SSO, audit logs, fine‑grained access.  
- **Scalable Collaboration** – Share code, demos, or entire projects with teammates worldwide.  
- **Fast Innovation Cycle** – Deploy new models and datasets instantly, iterate in minutes.

---

## 🏢 Culture & Vision

- **Community‑First** – 1 M+ users, 10 k+ contributors, open‑review model releases.  
- **Inclusive & Transparent** – Everyone can push, review, and learn from each other.  
- **Continuous Learning** – Hackathons, workshops, and research collaborations empower growth.  
- **Future‑Forward** – We’re building a unified AI stack – from data to inference to deployment.

---

## 👥 Careers & Growth

Hugging Face is looking for passionate AI/ML engineers, data scientists, product managers, and infrastructure specialists. We value curiosity, ownership, and collaboration. Whether you’re building next‑gen NLP models, scaling inference workloads, or designing community tools, there’s a place for you.

*Explore open roles and apply today on our Careers page.*

---

## 📞 Get Involved

- **Join the community** – Sign up for a free account, explore 1M + models, start a Space.  
- **Try Enterprise** – Contact our sales team for a tailored demo and pricing.  
- **Stay informed** – Subscribe to our newsletter, join Discord, follow us on Twitter, and attend Hugging Face Hackathons.

---

*Hugging Face – The platform where the machine‑learning community collaborates on models, datasets, and applications. Together, we’re building the AI future.*

In [33]:
stream_brochure("Cursor", "https://cursor.com")

# Cursor – The Future of Coding

---

## **About Cursor**

*Built to make you extraordinarily productive, Cursor is the best way to code with AI.*  
Cursor fuses a smart IDE, a powerful command‑line assistant, and a mobile agent into one seamless toolkit. With an artistic, landscape‑backed interface and real‑time AI suggestions, coding becomes faster, cleaner, and more enjoyable.

---

## **Key Features**

| Tool | What It Does | Benefit |
|------|--------------|---------|
| **AI‑Powered IDE** | Auto‑complete, refactor, and debug code while you type | Write correct code faster |
| **CLI Assistant** | Context‑aware help for scripts, commands, and workflows | No more terminal hunting |
| **Mobile Agent** | Code on the go, receive suggestions anywhere | Keep productivity flowing |
| **Rules & Policies** | Fine‑tune assistant behavior with custom rules | Tailor the AI to your team’s style |
| **Enterprise‑Ready** | Order‑management demos, analytics, and security | Scale without compromise |

An interactive demo shows each interface in action – the AI assistant writing a PyTorch experiment, a mobile agent answering questions, and the CLI helping with deployment tasks, all against a stunning painted landscape backdrop.

---

## **Enterprise Solutions**

Enterprise customers get:

- **Dedicated Support** – 24/7 help for production deployments
- **Analytics** – Track agent usage patterns and productivity gains
- **Compliance & Security** – ISO‑27001 compliant, data residency options
- **Integration** – Seamless with GitHub, GitLab, Slack, and custom pipelines

Whether you’re building an order management system or training machine‑learning models, Cursor’s AI accelerates entire workflows.

---

## **Company Culture**

- **Developer‑First Mindset** – Every feature is built with the everyday programmer in mind.
- **Innovation at Speed** – Rapid release cycle, community‑driven feedback loop.
- **Data‑Driven** – Decisions guided by usage analytics and user research.
- **Collaboration** – Open forums, community chat, and a vibrant documentation ecosystem.
- **Inclusive** – Diverse team, flexible work arrangements, continuous learning.

Cursor isn’t just a tool—it’s a community that thrives on building smarter code together.

---

## **Careers at Cursor**

> *“If you love coding, AI, and making developers happier, Cursor is the place for you.”*

Positions currently open (or frequently open):

| Role | Why It’s Important |
|------|--------------------|
| **Software Engineer – IDE** | Build the core assistant that developers rely on. |
| **AI Research Engineer** | Advance the models that power suggestions and completions. |
| **Product Manager** | Shape the roadmap and define user experiences. |
| **DevOps/Infrastructure** | Ensure high‑availability, security, and scaling. |
| **Community Lead** | Grow and nurture the developer community. |

**Apply** through our careers page or share your portfolio with us. We’re looking for passionate engineers, researchers, and creators.

---

## **Get Started**

- **Download the Mac app** – try Cursor free for today’s developers.
- **Explore the demo** – see the AI IDE and CLI in action right on the landing page.
- **Join the community** – find help, share tips, or contribute.

> *“Cursor lets us focus on architecture instead of boilerplate. It’s a game‑changer.”* – Happy Developer

---

### **Contact**

- **Website**: www.cursor.dev  
- **Email**: support@cursor.dev  
- **Slack**: Join the `#help` channel on the public Slack workspace.

---

**Cursor – code faster, think bigger, build better.**